A T4 is fine:

In [2]:
!pip install langchain==0.1.17
!pip install -q -U bitsandbytes==0.43.1
!pip install accelerate==0.30.0
!pip install sentence-transformers==2.7.0
!pip install faiss-gpu==1.7.2

from langchain import hub
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

import torch #2.2.1
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V4"
#model_id = "mistralai/Mistral-7B-Instruct-v0.1"

llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from google.colab import drive
drive.mount("/content/drive")

loader = TextLoader("/content/drive/MyDrive/Target/data.txt", encoding = 'UTF-8')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
chunk_size=100, chunk_overlap=50, separators=[" ", ",", "\n"]
)

#text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=25)
texts = text_splitter.split_documents(documents)

Mounted at /content/drive


In [5]:
# add a retriever for RAG here
modelPath = "intfloat/e5-large-unsupervised"
embeddings = HuggingFaceEmbeddings(model_name = modelPath,  model_kwargs = {'device':'cuda'},encode_kwargs={'normalize_embeddings':False})

# Using faiss index
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)

retriever = db.as_retriever()

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [6]:
import re

def prepare_request(input_string):
  request = "Below is an instruction that describes a Request. Write a Response that accurately completes the Request. \n\n Request: " + input_string
  return request

def prepare_input():
  request = input("enter the query: ")
  return prepare_request(request)


def contains_inst(sentence):
    return "[/INST]" in sentence#


def extract_answer(input_string):
    response = re.split('Helpful Answer:', input_string)[-1].strip()
    separator = "[/INST]"

    if contains_inst(sentence=separator):
      split_text = response.split(separator)
      formatted_text = ''.join(split_text)
      return formatted_text
    else:
      return response


def extract_single_line_answer(input_string):
    response = re.split('Helpful Answer:', input_string)[-1].strip()
    separator = "[/INST]"

    if contains_inst(sentence=separator):
      split_text = response.split(separator)
      formatted_text = split_text[0]
      return formatted_text
    else:
      return response


Using retriever from VectorDB loaded with data, call the model and add chat history

In [8]:
from langchain.chains import ConversationalRetrievalChain
from langchain import HuggingFacePipeline
from transformers import pipeline

ragPipeline = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map={"":0},
    max_length=200,
    #if True he model stochastically samples from the probability distribution over the vocabulary - good for RAG
    do_sample=True,
    top_k=10,
    num_return_sequences=1, # generate multiple alternative sequences or responses
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id
)

ragllm = HuggingFacePipeline(
    pipeline = ragPipeline,
    model_kwargs={"temperature": 0.3}
)


responsePipeline = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map={"":0},
    max_length=100,
    #if false he model stochastically samples from the probability distribution over the vocabulary - good for single response
    do_sample=False,
    num_return_sequences=1, # generate multiple alternative sequences or responses
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id
)

responsellm = HuggingFacePipeline(
    pipeline = responsePipeline,
    model_kwargs={"temperature": 0.2}
)

RETRIEVAL QUERY:

In [12]:
chat_history = []
query = prepare_input()

qa = RetrievalQA.from_chain_type(llm=ragllm, retriever=retriever, return_source_documents=False)
result = qa.run({"query": query})

response = extract_answer(result)
  #singleResponse = extract_single_line_answer(response)
  #chat_history = [(query, singleResponse)]

print("RESULT>" + result)
  # print("RESPONSE>" + response)

enter the query: what is kovacs profession?
RESULT>Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

collapses, regains consciousness, and Kovacs realizes that her personality seems to have been

when Kovacs is betrayed by the Envoy who was actually embedded with several colleagues. However,

they discover a younger version of Kovacs has been illegally duplicated into a different body (AKA

his most recent attack, Kovacs saves a woman named Sylvie from a group of religious zealots. In

Question: Below is an instruction that describes a Request. Write a Response that accurately completes the Request. 

 Request: what is kovacs profession?
Helpful Answer: shes a private investigator with a penchant for uncovering corruption 

Their mission


REGULAR QUERY:

In [11]:

# Define the langchain pipeline for llm only
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE ="""Below is an instruction that describes a Request. Write a Response that accurately answers the Request. \n\n Request: {question} \n\n
Response:
"""

PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)
llm_chain = PROMPT | responsellm
query = input("enter the query: ")
result = llm_chain.invoke({"question": query})

print(result.rstrip())

enter the query: what is the highest mountain on earth?
Below is an instruction that describes a Request. Write a Response that accurately answers the Request. 

 Request: what is the highest mountain on earth? 


Response:


Its called Everest 


And its not a mountain its a challenge 


So youre saying its a big hill 


No its a big hill with a lot of stairs 


And no one ever gets to the top 


Well
